<a href="https://colab.research.google.com/github/waqqasansari/Natural_Language_Processing/blob/master/QouraQP_Siamese_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"waqqas","key":"5985e115491a47485587329f9879e89c"}'}

In [ ]:
!mkdir p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c quora-question-pairs

 92% 105M/114M [00:01<00:00, 52.3MB/s] 
100% 114M/114M [00:01<00:00, 65.8MB/s]
 43% 9.00M/21.2M [00:00<00:00, 15.3MB/s]
100% 21.2M/21.2M [00:00<00:00, 31.0MB/s]
100% 4.95M/4.95M [00:00<00:00, 33.2MB/s]

test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!pip install -q -U trax

In [ ]:
import os
import nltk
import trax
from trax import layers as tl
from trax.supervised import training
from trax.fastmath import numpy as fastnp
import numpy as np
import pandas as pd
import random as rnd
nltk.download('punkt')
# set random seeds
rnd.seed(34)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
!unzip /content/train.csv.zip

Archive:  /content/train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
!unzip /content/test.csv.zip

Archive:  /content/test.csv.zip
  inflating: test.csv                


In [ ]:
data = pd.read_csv('train.csv')
N = len(data)
print('Number of question pairs:', N)
data.head(10)

Number of question pairs: 404290


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


In [ ]:
N_train = 300000
N_test = 10*1024
data_train = data[:N_train]
data_test = data[N_train : N_train+N_test]
del(data) #remove to free memory

In [ ]:
print("Train set:", len(data_train), "Test set:", len(data_test))

Train set: 300000 Test set: 10240


we only select question pair that are duplicate to train the model

In [ ]:
td_index = (data_train['is_duplicate'] == 1).to_numpy()
td_index = [i for i, x in enumerate(td_index) if x]
print('number of duiplicate questions: ', len(td_index))
print('indexes of first ten duplicate questions:', td_index[:10])

number of duiplicate questions:  111473
indexes of first ten duplicate questions: [5, 7, 11, 12, 13, 15, 16, 18, 20, 29]


In [ ]:
print(data_train['question1'][5])
print(data_train['question2'][5])
print('is_duplicate: ', data_train['is_duplicate'][5])

Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?
I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?
is_duplicate:  1


In [ ]:
Q1_train_words = np.array(data_train['question1'][td_index])
Q2_train_words = np.array(data_train['question2'][td_index])

Q1_test_words = np.array(data_test['question1'])
Q2_test_words = np.array(data_test['question2'])
y_test  = np.array(data_test['is_duplicate'])

In [ ]:
print('TRAINING QUESTIONS:\n')
print('Question 1: ', Q1_train_words[0])
print('Question 2: ', Q2_train_words[0], '\n')
print('Question 1: ', Q1_train_words[5])
print('Question 2: ', Q2_train_words[5], '\n')

print('TESTING QUESTIONS:\n')
print('Question 1: ', Q1_test_words[0])
print('Question 2: ', Q2_test_words[0], '\n')
print('is_duplicate =', y_test[0], '\n')

TRAINING QUESTIONS:

Question 1:  Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?
Question 2:  I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me? 

Question 1:  What would a Trump presidency mean for current international master’s students on an F1 visa?
Question 2:  How will a Trump presidency affect the students presently in US or planning to study in US? 

TESTING QUESTIONS:

Question 1:  What were some of the troubles you have faced during and after your 9 months period of pregnancy?
Question 2:  What is the difference between neural circuit and neural system? 

is_duplicate = 0 



In [ ]:
#create arrays
Q1_train = np.empty_like(Q1_train_words)
Q2_train = np.empty_like(Q2_train_words)

Q1_test = np.empty_like(Q1_test_words)
Q2_test = np.empty_like(Q2_test_words)

In [ ]:
# Building the vocabulary with the train set
from collections import defaultdict

vocab = defaultdict(lambda: 0)
vocab['<PAD>'] = 1

for idx in range(len(Q1_train_words)):
  Q1_train[idx] = nltk.word_tokenize(Q1_train_words[idx])
  Q2_train[idx] = nltk.word_tokenize(Q2_train_words[idx])
  q = Q1_train[idx] + Q2_train[idx]
  for word in q:
    if word not in vocab:
      vocab[word] = len(vocab) + 1
print('The length of the vocabulary is:', len(vocab))

The length of the vocabulary is: 36352


In [ ]:
print(vocab['<PAD>'])
print(vocab['Astrology'])
print(vocab['Astronomy'])  #not in vocabulary, returns 0

1
2
0


In [ ]:
for idx in range(len(Q1_test_words)):
  Q1_test[idx] = nltk.word_tokenize(Q1_test_words[idx])
  Q2_test[idx] = nltk.word_tokenize(Q2_test_words[idx])

In [ ]:
print('Train set has reduced to: ', len(Q1_train) ) 
print('Test set length: ', len(Q1_test) ) 

Train set has reduced to:  111473
Test set length:  10240


In [ ]:
# Converting questions to array of integers
for i in range(len(Q1_train)):
    Q1_train[i] = [vocab[word] for word in Q1_train[i]]
    Q2_train[i] = [vocab[word] for word in Q2_train[i]]
          
for i in range(len(Q1_test)):
    Q1_test[i] = [vocab[word] for word in Q1_test[i]]
    Q2_test[i] = [vocab[word] for word in Q2_test[i]]

In [ ]:
print('first question in the train set:\n')
print(Q1_train_words[0], '\n') 
print('encoded version:')
print(Q1_train[0],'\n')

print('first question in the test set:\n')
print(Q1_test_words[0], '\n')
print('encoded version:')
print(Q1_test[0]) 

first question in the train set:

Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me? 

encoded version:
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21] 

first question in the test set:

What were some of the troubles you have faced during and after your 9 months period of pregnancy? 

encoded version:
[30, 271, 116, 131, 78, 28882, 53, 218, 6595, 124, 11, 267, 56, 1636, 606, 2091, 131, 4329, 21]


In [ ]:
# Splitting the data
cut_off = int(len(Q1_train)*.8)
train_Q1, train_Q2 = Q1_train[:cut_off], Q2_train[:cut_off]
val_Q1, val_Q2 = Q1_train[cut_off: ], Q2_train[cut_off:]
print('Number of duplicate questions: ', len(Q1_train))
print("The length of the training set is:  ", len(train_Q1))
print("The length of the validation set is: ", len(val_Q1))

Number of duplicate questions:  111473
The length of the training set is:   89178
The length of the validation set is:  22295


In [ ]:
def data_generator(Q1, Q2, batch_size, pad=1, shuffle=True):


    input1 = []
    input2 = []
    idx = 0
    len_q = len(Q1)
    question_indexes = [*range(len_q)]
    
    if shuffle:
        rnd.shuffle(question_indexes)
    

    while True:
        if idx >= len_q:
            idx = len_q
            if shuffle:
                rnd.shuffle(question_indexes)
        
        q1 = Q1[question_indexes[idx]]
        q2 = Q2[question_indexes[idx]]
        
        idx += 1

        input1.append(q1)
        input2.append(q2)
        if len(input1) == batch_size:
            max_len = max(max([len(q) for q in input1]),
                          max([len(q) for q in input2]))
            max_len = 2**int(np.ceil(np.log2(max_len)))
            b1 = []
            b2 = []
            for q1, q2 in zip(input1, input2):
                q1 = q1 + [pad] * (max_len - len(q1))
                q2 = q2 + [pad] * (max_len - len(q2))
                b1.append(q1)
                b2.append(q2)
            yield np.array(b1), np.array(b2)
            input1, input2 = [], []

In [ ]:
b = [1,2,4,23,235,3,25,235,25,324,46,54,2]
a = len(b)
c = [*range(a)]
f = b[c[0]]
print(c)
print(a)
print(f)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
13
1


In [ ]:
batch_size = 64
res1, res2 = next(data_generator(train_Q1, train_Q2, batch_size))
print('First queation :', '\n', res1, '\n')
print('seciond question :', '\n',res2)

First queation : 
 [[  30   87  116 ...    1    1    1]
 [8843  572   39 ...    1    1    1]
 [ 283  111  156 ...    1    1    1]
 ...
 [ 168  905   62 ...    1    1    1]
 [ 356   78 7227 ...    1    1    1]
 [  32   38   53 ...    1    1    1]] 

seciond question : 
 [[  30   87   78 ...    1    1    1]
 [  30   87   78 ...    1    1    1]
 [  30  156   78 ...    1    1    1]
 ...
 [  32   76  905 ...    1    1    1]
 [ 356   78 7227 ...    1    1    1]
 [  32   33    4 ...    1    1    1]]


In [ ]:
def Siamese(vocab_size=len(vocab), d_model=128, mode='train'):
  def normalize(x):
    return x / fastnp.sqrt(fastnp.sum(x * x, axis=-1, keepdims = True))

  q_processor = tl.Serial(
      tl.Embedding(vocab_size, d_model),
      tl.LSTM(d_model),
      tl.Mean(axis=1),
      tl.Fn('Normalize', lambda x : normalize(x)) 
  )

  model = tl.Parallel(q_processor, q_processor)
  return model

In [ ]:
# check your model
model = Siamese()
print(model)

Parallel_in2_out2[
  Serial[
    Embedding_41794_128
    LSTM_128
    Mean
    Normalize
  ]
  Serial[
    Embedding_41794_128
    LSTM_128
    Mean
    Normalize
  ]
]


In [ ]:
def TripletLossFn(v1, v2, margin=0.25):
  scores = fastnp.dot(v1, v2.T)
  batch_size = len(scores)
  positive = fastnp.diagonal(scores)
  negative_without_positive = scores - 2.0 * fastnp.eye(batch_size)
  closest_negative = negative_without_positive.max(axis=1)
  negative_zero_on_duplicate = scores * (1.0 - fastnp.eye(batch_size))
  mean_negative = np.sum(negative_zero_on_duplicate, axis=1) / (batch_size-1)
  triplet_loss1 = fastnp.maximum(0.0, margin - positive + closest_negative)
  triplet_loss2 = fastnp.maximum(0.0, margin - positive + mean_negative)
  triplet_loss = fastnp.mean(triplet_loss1 + triplet_loss2)
  return triplet_loss

In [ ]:
v1 = np.array([[0.26726124, 0.53452248, 0.80178373],[0.5178918 , 0.57543534, 0.63297887]])
v2 = np.array([[ 0.26726124,  0.53452248,  0.80178373],[-0.5178918 , -0.57543534, -0.63297887]])
TripletLossFn(v2,v1)
print("Triplet Loss:", TripletLossFn(v2,v1))

Triplet Loss: 0.5


In [ ]:
from functools import partial
def TripletLoss(margin=0.25):
    triplet_loss_fn = partial(TripletLossFn, margin=margin)
    return tl.Fn('TripletLoss', triplet_loss_fn)

In [ ]:
batch_size = 128
train_generator = data_generator(train_Q1, train_Q2, batch_size, vocab['<PAD>'])
val_generator = data_generator(val_Q1, val_Q2, batch_size, vocab['<PAD>'])
print('train_Q1.shape ', train_Q1.shape)
print('val_Q1.shape   ', val_Q1.shape)

train_Q1.shape  (89178,)
val_Q1.shape    (22295,)


In [ ]:
lr_schedule = trax.lr.warmup_and_rsqrt_decay(400, 0.01)
def train_model(Siamese, TripletLoss,lr_schedule, train_generator=train_generator, 
                val_generator=val_generator, output_dir='model/'):

  output_dir = os.path.expanduser(output_dir)

  train_task = training.TrainTask(
        labeled_data = train_generator,         # Use generator (train)
        loss_layer = TripletLoss(),             # Use triplet loss. Don't forget to instantiate this object
        optimizer = trax.optimizers.Adam(0.01), # Don't forget to add the learning rate parameter
        lr_schedule=lr_schedule,              # Use Trax multifactor schedule function
        n_steps_per_checkpoint = 10
    )

  eval_task = training.EvalTask(
        labeled_data = val_generator,       # Use generator (val)
        metrics = [TripletLoss()])          # Use triplet loss. Don't forget to instantiate this object

  training_loop = training.Loop(Siamese(),
                                  train_task,
                                  eval_tasks = [eval_task],
                                  output_dir = output_dir)

  return training_loop

In [ ]:
train_steps = 500
training_loop = train_model(Siamese, TripletLoss, lr_schedule)
training_loop.run(train_steps)


Step      1: Ran 1 train steps in 3.16 secs
Step      1: train TripletLoss |  0.49693477
Step      1: eval  TripletLoss |  0.49887902

Step     10: Ran 9 train steps in 4.47 secs
Step     10: train TripletLoss |  0.49902451
Step     10: eval  TripletLoss |  0.49959910

Step     20: Ran 10 train steps in 1.73 secs
Step     20: train TripletLoss |  0.49791786
Step     20: eval  TripletLoss |  0.49721932

Step     30: Ran 10 train steps in 1.75 secs
Step     30: train TripletLoss |  0.48867178
Step     30: eval  TripletLoss |  0.49656379

Step     40: Ran 10 train steps in 3.69 secs
Step     40: train TripletLoss |  0.48058835
Step     40: eval  TripletLoss |  0.42655259

Step     50: Ran 10 train steps in 1.85 secs
Step     50: train TripletLoss |  0.43212983
Step     50: eval  TripletLoss |  0.39185748

Step     60: Ran 10 train steps in 1.88 secs
Step     60: train TripletLoss |  0.39067763
Step     60: eval  TripletLoss |  0.40240365

Step     70: Ran 10 train steps in 1.87 secs
Step

In [ ]:
# Loading in the saved model
model = Siamese()
model.init_from_file('/content/model/model.pkl.gz')

In [ ]:
def classify(test_Q1, test_Q2, y, threshold, model, vocab, data_generator=data_generator, batch_size=64):
  accuracy = 0
  for i in range(0, len(test_Q1), batch_size):
    q1, q2 = next(data_generator(test_Q1[i:i + batch_size], test_Q2[i:i + batch_size], batch_size, vocab['<PAD>'], shuffle=False))
    y_test = y[i:i + batch_size]
    v1, v2 =model((q1, q2))
    for j in range(batch_size):
      d = np.dot(v1[j], v2[j].T)
      res = d > threshold
      accuracy += (y_test[j] == res)
  accuracy = accuracy / len(test_Q1)
  return accuracy

In [ ]:
accuracy = classify(Q1_test, Q2_test, y_test, 0.7, model, vocab, batch_size = 512) 
print("Accuracy", accuracy)

Accuracy 0.70654296875


In [ ]:
def predict(question1, question2, threshold, model, vocab, data_generator=data_generator, verbose=False):
  q1 = nltk.word_tokenize(question1) 
  q2 = nltk.word_tokenize(question2)

  Q1, Q2 = [], []
  for word in q1:
    Q1 += [vocab[word]]
  for word in q2:
    Q2 += [vocab[word]]

  Q1, Q2 = next(data_generator([Q1], [Q2], 1, vocab['<PAD>']))
  v1, v2 = model((Q1, Q2))
  d = np.dot(v1[0], v2[0].T)
  res = d > threshold

  if(verbose):
    print("Q1  = ", Q1, "\nQ2  = ", Q2)
    print("d   = ", d)
    print("res = ", res)

  return res

In [ ]:
question1 = "When will I see you?"
question2 = "When can I see you again?"
# 1 means it is duplicated, 0 otherwise
predict(question1 , question2, 0.7, model, vocab, verbose = True)

Q1  =  [[585  76   4  46  53  21   1   1]] 
Q2  =  [[ 585   33    4   46   53 7287   21    1]]
d   =  0.8535244
res =  True


True

In [ ]:
question1 = "I had mad fun at club"
question2 = "had mad fun at club"
# 1 means it is duplicated, 0 otherwise
predict(question1 , question2, 0.7, model, vocab, verbose = True)

Q1  =  [[    4   813 18210  4566   129 10501     1     1]] 
Q2  =  [[  813 18210  4566   129 10501     1     1     1]]
d   =  0.8329049
res =  True


True